In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout, Output, Label
from IPython.display import display, Latex, Image, Markdown

import params as st
from model import linSys, nonlinSys
from feedforward import *
from observer import linObs

import numpy as np
from scipy.signal import place_poles
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

# Regelung des Brückenkrans

In [ ]:
imag = Image("../images/craneNonLin.png", width=250)

outL = Output()
outR = Output()

with outL:
    display(imag)
with outR:
    display(Markdown("""
**Modellgleichungen** mit Störung $\\Delta u$ im Eingang

nichtlinear |   | linear
----------------------|----|------------------------
\\begin{align*}
    M \\ddot{D}(t) & = u(t) + \\Delta u + F(t) \\langle\\mathbf{\\eta}(t),\\mathbf{e}_2 \\rangle \\\\
    m (\\ddot{\\mathbf{y}}(t) - \\mathbf{g}) & = -F(t) \\mathbf{\\eta}(t) \\\\
    \\mathbf{y}(t) &  = D(t) \\mathbf{e}_2 + l \\mathbf{\\eta}(t)
\\end{align*} |   | \\begin{align*}
    M \\ddot{D}(t) & = u(t) + \\Delta u + m g \\theta(t) \\\\
    m \\ddot{y}(t) & = -m g \\theta(t) \\\\
    y(t) &  = D(t) + l \\theta(t)
\\end{align*}

**flachheitsbasierte Steuerung mit Zustandsregler auf Basis des linearen Modells**

\\begin{align*}
    u_{\\mathrm{r}}(t) & = \\frac{M l}{g}  y_{\\mathrm{r}}^{(4)}(t) + (M + m)  \\ddot{y}_{\\mathrm{r}}(t) \\\\ 
    \\tilde{u}(t) & = u_{\\mathrm{r}}(t) - a_3 (y^{(3)}_{\\mathrm{r}}(t) - y^{(3)}(t)) - a_2 (\\ddot{y}_{\\mathrm{r}}(t) - \\ddot{y}(t)) - a_1 (\\dot{y}_{\\mathrm{r}}(t) - \\dot{y}(t)) - a_0 (y_{\\mathrm{r}}(t) - y(t)),\\\\
    & \\quad a_3,a_2,a_1,a_0 > 0
\\end{align*}


**Störgrößenkompensation mittels Beobachter**
\\begin{align*}
    \\dot{\\hat{\\mathbf{x}}}(t) & = \\begin{bmatrix}
    0 & 1 & 0 & 0 & 0\\\\
    0 & 0 & \\frac{g m}{M} & 0 & \\frac{1}{M} \\\\
    0 & 0 & 0 & 1 & 0 \\\\
    0 & 0 & - \\frac{g}{l}-\\frac{g m}{M l} & 0 & -\\frac{1}{M l}\\\\
    0 & 0 & 0 & 0 & 0
    \\end{bmatrix} \\hat{\\mathbf{x}}(t) + \\begin{bmatrix}
    0 \\\\  \\frac{1}{M} \\\\ 0 \\\\ -\\frac{1}{M l} \\\\ 0
    \\end{bmatrix} u(t) + \\mathbf{l} (\\hat{y}(t) - y(t)), & \\hat{\\mathbf{x}}(t) & = \\begin{bmatrix}
    \\hat{D}(t) \\\\ \\dot{\\hat{D}}(t) \\\\ \\hat{\\theta}(t) \\\\ \\dot{\\hat{\\theta}}(t) \\\\ z(t)
    \\end{bmatrix} \\\\
    \\hat{y}(t) & = \\begin{bmatrix} 1 & 0 & l & 0 & 0 \\end{bmatrix}\\hat{\\mathbf{x}}(t) 
\\end{align*}
mit Eingang
\\begin{align*}
    u(t) & = \\tilde{u}(t) - z(t)
\\end{align*}

    """))
cols = HBox([outL, outR], layout=Layout(display='flex', flex_flow='row', justify_content='space-around', align_items='center'))
display(cols)

**Lineares Systemgleichungen**

In [ ]:
A = sp.Matrix([[0, 1, 0, 0], [0, 0, st.m / st.M * st.g, 0], [0, 0, 0, 1], [0, 0, -st.g / st.l - st.m * st.g / (st.M * st.l), 0]])
b = sp.Matrix([[0], [1/st.M], [0], [-1/ (st.M * st.l)]])
c = sp.Matrix([[1], [0], [st.l], [0]])
n = A.shape[0]
display(Latex("$A = {},\\quad b = {}, \\quad c^{{\\intercal}} = {}$".format(sp.latex(A), sp.latex(b), sp.latex(c.T))))
_A = np.array(A).astype(np.float64)
_b = np.array(b).astype(np.float64)

**Steuerbarkeit des Systems**

In [ ]:
S = b
for i in range(1, n):
    S = S.row_join(A ** i * b)
display(Latex("$S = {}$".format(sp.latex(S))))
display(Latex("$rang(S) = {} = n = {}$".format(sp.latex(S.rank()), sp.latex(n))))

In [ ]:
t1 = sp.eye(4)[-1, :] * S ** -1
T = t1
for i in range(1, n):
    T = T.col_join(t1 * A ** i)
display(Latex("$T = {}$".format(sp.latex(T))))
_T = np.array(T).astype(np.float64)

**Beobachtbarkeit des Systems**

In [ ]:
O = c.T
for i in range(1, n):
    O = O.col_join(c.T * A ** i)
display(Latex("$O = {}$".format(sp.latex(O))))
display(Latex("$rang(O) = {} = n = {}$".format(sp.latex(O.rank()), sp.latex(n))))

**Eigenwerte des Systems**

In [ ]:
A.eigenvals()

**Definition Parameter**

In [ ]:
tSim = np.linspace(0, 10, 1001)

In [ ]:
output = widgets.Output()

with output:
    fig = plt.figure(figsize=(10, 5))
    ax3 = plt.subplot(222)
    ax4 = plt.subplot(224)
    ax1 = plt.subplot(221)
    ax2 = plt.subplot(223)

plt.subplots_adjust(wspace=0.2, hspace=0.3)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax11 = ax1.twinx()
ax1.set_xlim([tSim[0], tSim[-1]]) 
ax11.set_xlim([tSim[0], tSim[-1]]) 
ax2.set_xlim([tSim[0], tSim[-1]]) 
ax4.set_xlim([tSim[0], tSim[-1]]) 
ax1.set_ylim([-15, 15]) 
ax2.set_ylim([-0.1, 1.5]) 
ax4.set_ylim([-20, 20]) 
ax1.grid() 
ax2.grid()
ax4.grid()
ax3.set_xlim(-0.3, 1.3)
ax3.set_ylim(-st.l - 2 * st.r, 2 * st.h)
ax3.set_xticks([])
ax3.set_yticks([])
ax1.set_ylabel(r"$u$")
ax2.set_xlabel(r"$t$ in s")
ax2.set_ylabel(r"$y$ in m")
ax4.set_ylabel(r"$\theta$ in °")
ax4.set_xlabel(r"$t$ in s")

lineY, = ax2.plot([], [], label=r"Istverlauf")
lineYref, = ax2.plot([], [], '--', label=r"Sollverlauf")
lineYobs, = ax2.plot([], [], '-.', label=r"beobachteter Verlauf")
lineTheta, = ax4.plot([], [])
lineThetaref, = ax4.plot([], [], '--')
lineThetaobs, = ax4.plot([], [], '-.')
lineU, = ax1.plot([], [])
lineUref, = ax1.plot([], [], '--')
linedU, = ax11.plot([], [], 'C2-.')

pathAni = ax3.plot([-0.3, 1.3], [0, 0], zorder=0, color='C6')
seilAni, = ax3.plot([0, 0], [0, -st.l], zorder=0, color='0')
lastAni = ax3.add_patch(plt.Circle((0,-st.l), st.r, color='0' , zorder=1))
wagenAni = ax3.add_patch(plt.Rectangle((-st.b / 2, 0), st.b, st.h, facecolor='0.', edgecolor='0.'))

handlesAx, labelsAx = ax2.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.94, 0.7735, .15), loc=3,
           ncol=3, mode="expand", borderaxespad=0., framealpha=0.5)

playB = widgets.Play(value=0,
                     min=0, 
                     max=len(tSim),
                     step=10)
sliderB = widgets.IntSlider(value=0,
                            min=0,
                            max=len(tSim),
                            step=10)
sliderT0 = widgets.FloatSlider(value=0,
                               min=0,
                               max=3,
                               step=1,
                               description=r'$t_0$')
sliderT = widgets.FloatSlider(value=1.6,
                               min=1.5,
                               max=2,
                               step=.1,
                               description=r'$T$')
sliderYd = widgets.widgets.FloatSlider(value=1,
                                       min=0.7,
                                       max=1.2,
                                       step=0.1,
                                       description=r"$y_\text{d}$")
sliderDu = widgets.widgets.FloatSlider(value=0,
                                       min=-0.5,
                                       max=0.5,
                                       step=0.1,
                                       description=r"$\Delta u$")
sliderW0 = widgets.widgets.FloatSlider(value=1,
                                       min=0.5,
                                       max=1.2,
                                       step=0.05,
                                       description=r"$\omega_0$")
sliderD = widgets.widgets.FloatSlider(value=0.9,
                                      min=0,
                                      max=1,
                                      step=0.05,
                                      description=r"$D$")
sliderObs = widgets.Checkbox(value=False,
                             description='Aktiv',
                             disabled=False)
sliderObsD0 = widgets.widgets.FloatSlider(value=0,
                                          min=-0.2,
                                          max=0.2,
                                          step=0.1,
                                          description=r"$D_\text{0}$")
sliderSimD0 = widgets.widgets.FloatSlider(value=0,
                                          min=-0.2,
                                          max=0.2,
                                          step=0.1,
                                          description=r"$D_\text{0}$")
radioMod = widgets.RadioButtons(options=['linear', 'nichtlinear'],
                              disabled=False)

def updateOde(_):
    global res

    t0Ff = sliderT0.value
    TFf = sliderT.value
    ydFf = sliderYd.value
    obs = sliderObs.value
    du = sliderDu.value
    _D = sliderD.value
    _w0 = sliderW0.value
    D0Obs = sliderObsD0.value
    D0Sim = sliderSimD0.value
    sType = radioMod.value

    def sys(t, x, uFF, du, sType, obs, Dr, thetar, k, l, params, A, b, c):
        if obs:
            u = lambda t: uFF(t) - k @ (x[4:8] - np.array([Dr[0](t), Dr[1](t), thetar[0](t), thetar[1](t)])) + du - x[8]
            uObs = lambda t: uFF(t) - k @ (x[4:8] - np.array([Dr[0](t), Dr[1](t), thetar[0](t), thetar[1](t)])) - x[8]
        else:
            u = lambda t: uFF(t) - k @ (x[0:4] - np.array([Dr[0](t), Dr[1](t), thetar[0](t), thetar[1](t)])) + du
            uObs = lambda t: uFF(t) - k @ (x[0:4] - np.array([Dr[0](t), Dr[1](t), thetar[0](t), thetar[1](t)]))

        dx = np.zeros(9)
        # Model
        if sType == 'linear':
            dx[0:4] = linSys(t, x[0:4], u, params)
        else:
            dx[0:4] = nonlinSys(t, x[0:4], u, params)
        # Beobachter
        dx[4:9] = linObs(t, x[4:9], uObs, l, x[0:4], A, b, c)
        
        return dx

    params = st.g, st.M, st.m , st.l
    x0 = [D0Sim, 0, 0, 0, D0Obs, 0, 0, 0, 0]
    _, uFF, Dr, thetar = feedForwardLinFlat(t0Ff, TFf, ydFf)
    # Beobachterverstärkung
    A = np.array([[0, 1, 0, 0, 0], [0, 0, st.m / st.M * st.g, 0, 1 / st.M], [0, 0, 0, 1, 0], [0, 0, -st.g / st.l - st.m * st.g / (st.M * st.l), 0, -1 / (st.M * st.l)], [0, 0, 0, 0, 0]])
    b = np.array([[0], [1 / st.M], [0], [-1 / (st.M * st.l)], [0]])
    c = np.array([[1], [0], [st.l], [0], [0]])
    l = place_poles(A.T, c, np.array([-1, -2, -2.5, -2.75, -12])).gain_matrix[0]
    # Reglerverstärkung
    eigs = np.linalg.eigvals(_A)
    omega0 = np.abs(max(eigs)) * _w0
    k = place_poles(_A, _b, omega0 * np.array([-_D,-_D * 1.01, -_D + 1j * np.sqrt(1 - _D ** 2), -_D - 1j * np.sqrt(1 - _D **2)])).gain_matrix[0]
    res = solve_ivp(sys,
                    [tSim[0], tSim[-1]],
                    x0,
                    t_eval=tSim,
                    args=(uFF, du, sType, obs, Dr, thetar, k, l, params, A, b, c))

def updatePlot(change):
    idx = change['new']

    theta = res.y.T[idx, 2]
    D = res.y.T[idx, 0]
    y1 = D + np.sin(theta) * st.l 
    y2 = np.cos(theta) * st.l
    
    t0Ff = sliderT0.value
    TFf = sliderT.value
    ydFf = sliderYd.value

    du = sliderDu.value
    _D = sliderD.value
    _w0 = sliderW0.value
    obs = sliderObs.value
    D0Obs = sliderObsD0.value
    D0Sim = sliderSimD0.value
    
    yr, uFF, Dr, thetar = feedForwardLinFlat(t0Ff, TFf, ydFf)
    # Reglerverstärkung
    eigs = np.linalg.eigvals(_A)
    omega0 = np.abs(max(eigs)) * _w0
    k = place_poles(_A, _b, omega0 * np.array([-_D,-_D * 1.01, -_D + 1j * np.sqrt(1 - _D ** 2), -_D - 1j * np.sqrt(1 - _D **2)])).gain_matrix[0]

    if obs:
        u = lambda t, idx: uFF(t) - k @ (res.y.T[idx, 4:8] - np.array([Dr[0](t), Dr[1](t), thetar[0](t), thetar[1](t)])) + du - res.y.T[idx, 8]
    else:
        u = lambda t, idx: uFF(t) - k @ (res.y.T[idx, 0:4] - np.array([Dr[0](t), Dr[1](t), thetar[0](t), thetar[1](t)])) + du
    
    lineThetaref.set_data(tSim[:idx], [np.rad2deg(thetar[0](_t)) for _t in tSim[:idx]])
    lineYref.set_data(tSim[:idx], [yr[0](_t) for _t in tSim[:idx]])
    thrMin = np.min([np.rad2deg(thetar[0](_t)) for _t in tSim])
    thrMax = np.max([np.rad2deg(thetar[0](_t)) for _t in tSim])
    yrMin = np.min([yr[0](_t) for _t in tSim])
    yrMax = np.max([yr[0](_t) for _t in tSim])

    lineU.set_data(tSim[:idx], [u(_t, _i) for _i, _t in enumerate(tSim[:idx])])
    lineUref.set_data(tSim[:idx], [uFF(_t) for _t in tSim[:idx]])
    linedU.set_data(tSim[:idx], res.y.T[:idx, 8])
    lineTheta.set_data(tSim[:idx], np.rad2deg(res.y.T[:idx, 2]))
    lineThetaobs.set_data(tSim[:idx], np.rad2deg(res.y.T[:idx, 6]))
    lineY.set_data(tSim[:idx], res.y.T[:idx, 0] + np.sin(res.y.T[:idx, 2]) * st.l)
    lineYobs.set_data(tSim[:idx], res.y.T[:idx, 4] + np.sin(res.y.T[:idx, 6]) * st.l)

    duMin = np.min(res.y.T[:, 8])
    duMax = np.max(res.y.T[:, 8])
    uMin = np.minimum(np.min([uFF(_t) for _t in tSim]), np.min([u(_t, _i) for _i, _t in enumerate(tSim)]))
    uMax = np.maximum(np.max([uFF(_t) for _t in tSim]), np.max([u(_t, _i) for _i, _t in enumerate(tSim)]))
    thMin = np.minimum(np.minimum(np.min(np.rad2deg(res.y.T[:, 2])), np.min(np.rad2deg(res.y.T[:, 6]))), thrMin)
    thMax = np.maximum(np.maximum(np.max(np.rad2deg(res.y.T[:, 2])), np.max(np.rad2deg(res.y.T[:, 6]))), thrMax)    
    yMin = np.minimum(np.minimum(np.min(res.y.T[:, 0] + np.sin(res.y.T[:, 2]) * st.l), np.min(res.y.T[:, 4] + np.sin(res.y.T[:, 6]) * st.l)), yrMin)
    yMax = np.maximum(np.maximum(np.max(res.y.T[:, 0] + np.sin(res.y.T[:, 2]) * st.l), np.max(res.y.T[:, 4] + np.sin(res.y.T[:, 6]) * st.l)), yrMax)
    
    ax1.set_ylim(uMin - np.abs(uMax - uMin) * 0.1, uMax + np.abs(uMax - uMin) * 0.1)
    ax11.set_ylim(duMin - np.abs(duMax - duMin) * 0.1, duMax + np.abs(duMax - duMin) * 0.1)
    ax2.set_ylim(yMin - np.abs(yMax - yMin) * 0.1, yMax + np.abs(yMax - yMin) * 0.1)
    ax4.set_ylim(thMin - np.abs(thMax - thMin) * 0.1, thMax + np.abs(thMax - thMin) * 0.1)

    seilAni.set_data([D, y1] ,[0, -y2])
    lastAni.set_center((y1, -y2))
    wagenAni.set_x(D - st.b/2)
    
    fig.canvas.draw()    

sliderB.observe(updatePlot, names='value')
sliderT0.observe(updateOde, names='value')
sliderT.observe(updateOde, names='value')
sliderYd.observe(updateOde, names='value')
sliderObs.observe(updateOde, names='value')
sliderW0.observe(updateOde, names='value')
sliderDu.observe(updateOde, names='value')
sliderD.observe(updateOde, names='value')
sliderObsD0.observe(updateOde, names='value')
sliderSimD0.observe(updateOde, names='value')
radioMod.observe(updateOde, names='value')
    
updateOde(_)

bFeedForward = VBox([Label(value='Steuerung'), sliderYd, sliderT0, sliderT])
bFeedForward.layout = make_box_layout()
bFeedBack = VBox([Label(value='Regler'), sliderW0, sliderD])
bFeedBack.layout = make_box_layout()
bObserver = VBox([Label(value='Beobachter'), sliderObs, sliderObsD0])
bObserver.layout = make_box_layout()
bModel = VBox([Label(value='Modell'), radioMod, sliderSimD0, sliderDu])
bModel.layout = make_box_layout()
controls = HBox([bFeedForward, bFeedBack, bObserver, bModel])

jslink((playB, 'value'), (sliderB, 'value'))
videoControls = VBox([HBox([playB, sliderB]), output])
videoControls.layout = make_box_layout()

HBox([controls, Box([videoControls])], layout=Layout(display='flex', flex_flow='column', justify_content='center', align_items='center'))